<a href="https://colab.research.google.com/github/a-forty-two/DataSetsForML/blob/master/13_Word2Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# words = { the, day, looks , good } -> bag of words -> a set of words (note that order doesn't matter in set)
# yesterday-> classification using Bag or Words -> Spanish and English 

# the day looks ____ -> good 
# EMBEDDING 

# Idea behind vectorization to predict neighbor
# 1) Word pairs -> continuous bag of words!
# 2) Then divide this as training and testing data 
# 3) create a 3 layer network-> feed the word and predict the output! 
# 4) remove the last layer! keep the input/hidden layer
# 5) whatever weight recieved from hidden layer the new vector for the word!!! 

In [25]:
import numpy as np
import tensorflow as tf
corpus_raw = 'He is the rich guy. The rich guy is happy person. She is the rich girl.'
corpus_raw = corpus_raw.lower()
corpus_raw

'he is the rich guy. the rich guy is happy person. she is the rich girl.'

In [26]:
# a whole bag of words 
words = []
for word in corpus_raw.split():
  if word != '.':
    word = word.replace('.', '')
    words.append(word)

print(words)
print(len(words))
words = set(words)
print(words)
print(len(words))

['he', 'is', 'the', 'rich', 'guy', 'the', 'rich', 'guy', 'is', 'happy', 'person', 'she', 'is', 'the', 'rich', 'girl']
16
{'guy', 'she', 'person', 'girl', 'the', 'happy', 'rich', 'he', 'is'}
9


In [27]:
word2int = {}
int2word = {}
vocab_size = len(words)
for i,word in enumerate(words):
  word2int[word] = i
  int2word[i] = word 
print(word2int)
print(int2word)

{'guy': 0, 'she': 1, 'person': 2, 'girl': 3, 'the': 4, 'happy': 5, 'rich': 6, 'he': 7, 'is': 8}
{0: 'guy', 1: 'she', 2: 'person', 3: 'girl', 4: 'the', 5: 'happy', 6: 'rich', 7: 'he', 8: 'is'}


In [16]:
print(word2int['rich'])

7


In [17]:
print(int2word[3])

person.


In [28]:
# extract raw sentences from the corpus
rawSentences = corpus_raw.split('.') # . are end of sentences
sentences = []
for sentence in rawSentences:
  sentences.append(sentence.split())

sentences

[['he', 'is', 'the', 'rich', 'guy'],
 ['the', 'rich', 'guy', 'is', 'happy', 'person'],
 ['she', 'is', 'the', 'rich', 'girl'],
 []]

In [29]:
# to predict next word, given word, basically i need to create all possible word-sequence pairs
# longest word-sequnce we want to detect is 2-> Rich ___ ? GUY! 
WINDOW_SIZE = 2
data = []
# The algo below will create, all possible combinations of my words, within WINDOW RANGE
for sentence in sentences:
  for wordIndex, word in enumerate(sentence):
    for xword in sentence[max(wordIndex- WINDOW_SIZE,0):min(wordIndex+ WINDOW_SIZE,len(sentence))+1]: 
      if xword != word: # pair detected, check for self detection
        data.append([word,xword])

data

#wordIndex,word -> 0,Ba 1,ba 2,black 3,sheep
#wordIndex = 1, word = ba
#
#xword =Ba   0:3 -> FIRST 2 words will be selected, if they are different, they are considered word-pair
#word =ba    1:4 -> 2nd and 3rd words will be select, considered word-pair 
#            2:5
#            3:6
#            4:7....


[['he', 'is'],
 ['he', 'the'],
 ['is', 'he'],
 ['is', 'the'],
 ['is', 'rich'],
 ['the', 'he'],
 ['the', 'is'],
 ['the', 'rich'],
 ['the', 'guy'],
 ['rich', 'is'],
 ['rich', 'the'],
 ['rich', 'guy'],
 ['guy', 'the'],
 ['guy', 'rich'],
 ['the', 'rich'],
 ['the', 'guy'],
 ['rich', 'the'],
 ['rich', 'guy'],
 ['rich', 'is'],
 ['guy', 'the'],
 ['guy', 'rich'],
 ['guy', 'is'],
 ['guy', 'happy'],
 ['is', 'rich'],
 ['is', 'guy'],
 ['is', 'happy'],
 ['is', 'person'],
 ['happy', 'guy'],
 ['happy', 'is'],
 ['happy', 'person'],
 ['person', 'is'],
 ['person', 'happy'],
 ['she', 'is'],
 ['she', 'the'],
 ['is', 'she'],
 ['is', 'the'],
 ['is', 'rich'],
 ['the', 'she'],
 ['the', 'is'],
 ['the', 'rich'],
 ['the', 'girl'],
 ['rich', 'is'],
 ['rich', 'the'],
 ['rich', 'girl'],
 ['girl', 'the'],
 ['girl', 'rich']]

In [0]:
# convert from words to numbers! Label Encoder, One-Hot (INPUT), Label Binarizer(OUTPUT)!!! 
def Encoder(datapointindex, vocabsize):
  temp = np.zeros(vocabsize) # initialize with a cold vector
  temp[datapointindex] = 1 # wherever word is present, encode that position as hot
  return temp

trainx, trainy = [], []

In [31]:
# ONE-HOT Encoding
for worddata in data:
  try:
    trainx.append(Encoder(word2int[worddata[0]], vocab_size))
    trainy.append(Encoder(word2int[worddata[1]], vocab_size))
  except:
    continue 

trainx = np.asarray(trainx)
trainy = np.asarray(trainy)
print(trainx[:5])
print(trainy[:5])
print(len(trainx))
print(len(trainy))

[[0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]]
46
46


In [32]:
print(trainx.shape)
print(trainy.shape)

(46, 9)
(46, 9)


In [0]:
# BEGIN THE LEARNING PROCESS 
# Stop word removal may or may not be required, check if it is really required or can you do without it
# because every step takes time! 
# answer questions after reading a topic
# Manoj is good at singing. Pinky was good at singing. 
# is, was , at
# {manoj, good, singing} {pinky, good, singing}

# Q- Back in the days, who would sing good?

In [0]:
HP_embeddims = 5
EPOCHS = 100

x = tf.placeholder(tf.float32, shape=(None, vocab_size)) # infinite rows, dictionary size/column size is fixed
y = tf.placeholder(tf.float32, shape=(None, vocab_size))
w1 = tf.Variable(tf.random_normal([vocab_size, HP_embeddims]))
b1 = tf.Variable(tf.random_normal([HP_embeddims]))
hidden_rep = tf.add(tf.matmul(x,w1), b1) # hidden_rep = x*w1 + b
w2 = tf.Variable(tf.random_normal([HP_embeddims, vocab_size])) # truncated_normal -> between limits, random_normal
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_rep,w2), b2)) 
# every_layer = prev_layer*new_weights + new_bias
# 3 layered model! First layer->w1->Second->w2->Output


In [34]:
prediction # Pytorch -> like Numpy, was dynamic, and gave you answers immediately!
# tf.session() is required to run the code in tf!

<tf.Tensor 'Softmax:0' shape=(?, 9) dtype=float32>

In [0]:
session = tf.Session()
init = tf.global_variables_initializer() # THIS FUNCTION NEEDS TO BE EXECUTED VIA TF SESSION
session.run(init) # all global variables initiated



In [40]:
# LOSS FUNCTION
crossEntropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(prediction), reduction_indices=[1])) 
# FORMULA for cross entropy
# define the training step
trainStep = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(crossEntropy)
HP_epoch = 10000
# train the model
for _ in range(HP_epoch):
  session.run(trainStep, feed_dict={x:trainx, y:trainy})
  print("Loss")
  print(session.run(crossEntropy, feed_dict={x:trainx, y:trainy}))

Streaming output truncated to the last 5000 lines.
1.3902805
Loss
1.3902806
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902806
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902804
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902805
Loss
1.3902804
Loss
1.3902805
Loss
1.3902804
Loss
1.3902805
Loss
1.39

In [42]:
print(session.run(w2))
print("******")
print(session.run(w1 + b1))
print("******")
print(session.run(w2 + b2))
# p(y) = w * p(x) + bias = w + bias
# p(x) = 1! that's why i am predict y! p(x)-> probability a word is spoken! 

[[ 0.9843094   1.4250312   1.4274484  -0.51631415 -3.143081   -0.7861621
  -0.9010001   0.60109484 -0.06865055]
 [-0.27026778  1.2416583  -1.11245    -0.17458491 -0.18653247 -0.1411841
   3.0630634   0.9068032  -0.65290046]
 [-0.22337353  0.22471653  1.7307743  -1.8044176   0.5126649   3.1754405
   0.02360832 -0.07849466 -1.0573822 ]
 [-0.7896527   0.31177112 -0.3766726  -0.12773208  0.5360482  -0.3554552
   0.8359583   1.0009742  -2.7543576 ]
 [ 2.6445596   1.8529447   1.4421319   1.689314   -0.0059719  -1.7032994
  -0.05519323  0.9185902  -0.15526256]]
******
[[-1.7324152   1.883774    0.13550049 -2.0472891  -1.0058949 ]
 [-2.0666327  -0.49944556 -1.7790618  -1.3016596  -1.400016  ]
 [-0.49057007  0.28841868  1.0738897  -3.1505733  -1.2511703 ]
 [-2.5915198   2.3483486  -0.12705123 -0.7721926   0.93512756]
 [ 0.90756726  1.7640674  -2.4648957  -0.65195787  0.5537219 ]
 [ 1.0393269  -2.0281303  -0.10959721 -1.6155465   0.9968424 ]
 [-2.6770058  -0.05789685 -1.2792141  -2.365671    2.8

In [43]:
vectors = session.run(w1+b1) # this is the layer calculating the vectors for conversion!!!

print(vectors[word2int['happy']]) # LEARNING TRANSFERRED!!!! 

[ 1.0393269  -2.0281303  -0.10959721 -1.6155465   0.9968424 ]
